In [1]:
# Завантаження необхідних бібліотек і функцій
import numpy as np
from scipy.io import loadmat
from scipy.optimize import minimize

In [2]:
# Функція завантаження списку фільмів
def loadMovieList():
    with open('movie_ids.txt',  encoding='ISO-8859-1') as fid:
        movies = fid.readlines()
    movieNames = []
    for movie in movies:
        parts = movie.split()
        movieNames.append(' '.join(parts[1:]).strip())
    return movieNames

In [3]:
# Завантаження даних про рейтинги фільмів і матрицю індикаторів R та Y
data = loadmat('movies.mat')
Y = data['Y']  # Матриця рейтингів (num_movies x num_users)
R = data['R']  # Бінарна матриця індикаторів (num_movies x num_users)

In [5]:
# Ініціалізація параметрів
num_users = Y.shape[1]
num_movies = Y.shape[0]
num_features = 10  # Кількість ознак (зазвичай вибирається експериментально)

X = np.random.randn(num_movies, num_features)
Theta = np.random.randn(num_users, num_features)

In [6]:
# Функція втрат
def costFunction(params, Y, R, num_users, num_movies, num_features, lambda_):
    X = params[:num_movies * num_features].reshape(num_movies, num_features)
    Theta = params[num_movies * num_features:].reshape(num_users, num_features)

    predictions = X.dot(Theta.T)

    error = (predictions - Y)
    error = error * R
    squared_error = error ** 2
    J = 0.5 * np.sum(squared_error)

    reg_term = (lambda_ / 2) * (np.sum(Theta ** 2) + np.sum(X ** 2))
    J += reg_term

    return J

In [7]:
# Обчислення градієнтів
def gradient(params, Y, R, num_users, num_movies, num_features, lambda_):
    X = params[:num_movies * num_features].reshape(num_movies, num_features)
    Theta = params[num_movies * num_features:].reshape(num_users, num_features)

    error = (X.dot(Theta.T) - Y)
    error = error * R

    X_grad = error.dot(Theta) + lambda_ * X
    Theta_grad = error.T.dot(X) + lambda_ * Theta

    grad = np.concatenate([X_grad.ravel(), Theta_grad.ravel()])

    return grad

In [8]:
# Параметри оптимізації
lambda_ = 10

In [9]:
# Об'єднати матриці X та Theta в один вектор параметрів
initial_params = np.concatenate([X.ravel(), Theta.ravel()])

In [10]:
# Мінімізація функції втрат
result = minimize(costFunction, initial_params, args=(Y, R, num_users, num_movies, num_features, lambda_),
                  method='CG', jac=gradient, options={'maxiter': 100})

In [11]:
# Отримання оптимальних параметрів X та Theta
optimal_params = result.x
X = optimal_params[:num_movies * num_features].reshape(num_movies, num_features)
Theta = optimal_params[num_movies * num_features:].reshape(num_users, num_features)



Тепер X і Theta містять оптимальні значення, які можна використовувати для прогнозування рейтингів

In [14]:
user_idx = 2  # Замініть на індекс конкретного користувача

# Прогнози для користувача user_idx
user_predictions = X.dot(Theta[user_idx, :].T)

# Вибираємо фільми, які користувач ще не оцінив (де R[i, user_idx] == 0)
user_unrated_movies_predictions = user_predictions * (1 - R[:, user_idx])

# Отримуємо індекси фільмів, які мають найвищі прогнози (наприклад, топ-10 рекомендацій)
top_movie_indices = np.argsort(user_unrated_movies_predictions)[::-1][:10]

# Отримуємо назви рекомендованих фільмів
movie_names = loadMovieList()

print("Рекомендовані фільми для користувача {}:".format(user_idx))
for i, movie_idx in enumerate(top_movie_indices):
    print("{}. {}".format(i + 1, movie_names[movie_idx]))


Рекомендовані фільми для користувача 2:
1. Godfather, The (1972)
2. Star Wars (1977)
3. Apt Pupil (1998)
4. Clockwork Orange, A (1971)
5. Trainspotting (1996)
6. Big Lebowski, The (1998)
7. Boot, Das (1981)
8. Godfather: Part II, The (1974)
9. Bridge on the River Kwai, The (1957)
10. As Good As It Gets (1997)
